In [2]:
from test import device, model, context_input, response_input, embs_gen, cand_emb_gen, score

In [3]:
import pickle, os
import pandas as pd

if os.path.exists('./data/cand_embs.pickle'):
    with open('./data/cand_embs.pickle', 'rb') as f:
        cand_embs = pickle.load(f)
    cand_embs.to(device)
else:
    with open('./data/train_data_source.pickle', 'rb') as f:
        train = pickle.load(f)

    data = {
        'context' : [],
        'response': []
    }

    for sample in train:
        data['context'].append(sample['context'])
        data['response'].append([sample['responses'][0]])


    df = pd.DataFrame(data)

    ## generate cand_embs & create tensor table on device
    response_input_srs = df['response'].apply(response_input)
    response_input_lst = response_input_srs.to_list()

    cand_embs_lst = []
    for sample in response_input_lst:
        cand_embs_lst.append(cand_emb_gen(*sample).to('cpu'))

    cand_embs = cand_embs_lst[0]
    for idx in range(1, len(cand_embs_lst)):
        y = cand_embs_lst[idx]
        cand_embs = torch.cat((cand_embs, y), 1)
    cand_embs = cand_embs.to(device)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.